In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
# import json
# import sqlite3

class IndianDeliveryDataGenerator:
    def __init__(self):
        # Major Indian cities with their coordinates
        self.cities = {
            'Mumbai': {'lat': 19.0760, 'lon': 72.8777, 'traffic_factor': 1.8},
            'Delhi': {'lat': 28.6139, 'lon': 77.2090, 'traffic_factor': 1.7},
            'Bangalore': {'lat': 12.9716, 'lon': 77.5946, 'traffic_factor': 1.6},
            'Hyderabad': {'lat': 17.3850, 'lon': 78.4867, 'traffic_factor': 1.5},
            'Chennai': {'lat': 13.0827, 'lon': 80.2707, 'traffic_factor': 1.5},
            'Pune': {'lat': 18.5204, 'lon': 73.8567, 'traffic_factor': 1.4},
            'Kolkata': {'lat': 22.5726, 'lon': 88.3639, 'traffic_factor': 1.6}
        }

        # Indian vehicle types with capacity
        self.vehicle_types = {
            'Cargo Auto': {'capacity_weight': 500, 'capacity_volume': 5},
            'Tata Ace': {'capacity_weight': 750, 'capacity_volume': 8},
            'Mahindra Pickup': {'capacity_weight': 1200, 'capacity_volume': 12},
            'Delivery Bike': {'capacity_weight': 50, 'capacity_volume': 0.3},
            'Mini Truck': {'capacity_weight': 2500, 'capacity_volume': 20},
            'Large Truck': {'capacity_weight': 4000, 'capacity_volume': 32}
        }

        # Delivery priorities
        self.priority_levels = [
            'Standard', 
            'Express', 
            'Same Day', 
            'Next Day',
            'Premium'
        ]

        # Payment methods common in India
        self.payment_methods = [
            'Cash on Delivery',
            'UPI',
            'Credit Card',
            'Debit Card',
            'Net Banking',
            'Digital Wallet'
        ]

        # Traffic patterns specific to Indian cities
        self.traffic_patterns = {
            'morning_rush': {'hours': range(8, 11), 'multiplier_range': (1.5, 2.0)},
            'afternoon': {'hours': range(11, 16), 'multiplier_range': (1.2, 1.5)},
            'evening_rush': {'hours': range(16, 20), 'multiplier_range': (1.6, 2.2)},
            'night': {'hours': list(range(20, 24)) + list(range(0, 8)), 'multiplier_range': (0.8, 1.1)}
        }

        # Monsoon season affects (June to September)
        self.monsoon_months = [6, 7, 8, 9]

    def get_traffic_multiplier(self, timestamp, city):
        """Generate traffic multiplier based on time of day and city"""
        hour = timestamp.hour
        base_multiplier = self.cities[city]['traffic_factor']
        
        # Add monsoon effect
        if timestamp.month in self.monsoon_months:
            base_multiplier *= random.uniform(1.2, 1.5)
        
        for pattern, data in self.traffic_patterns.items():
            if hour in data['hours']:
                return base_multiplier * random.uniform(*data['multiplier_range'])
        
        return base_multiplier

    def generate_coordinates(self, base_city):
        """Generate coordinates within city limits"""
        base_lat = self.cities[base_city]['lat']
        base_lon = self.cities[base_city]['lon']
        
        # Generate point within ~10km radius
        radius = 0.1  # Approximately 10km
        r = radius * np.sqrt(random.random())
        theta = random.random() * 2 * np.pi
        
        lat = base_lat + r * np.cos(theta)
        lon = base_lon + r * np.sin(theta)
        
        return lat, lon

    def generate_time_window(self, base_time, priority):
        """Generate delivery time window based on priority"""
        time_windows = {
            'Same Day': (1, 4),     # 1-4 hours window
            'Express': (2, 6),      # 2-6 hours window
            'Premium': (1, 3),      # 1-3 hours window
            'Next Day': (8, 24),    # 8-24 hours window
            'Standard': (24, 48)    # 24-48 hours window
        }
        
        window_start, window_end = time_windows[priority]
        earliest_delivery = base_time + timedelta(hours=window_start)
        latest_delivery = base_time + timedelta(hours=window_end)
        
        return earliest_delivery, latest_delivery

    def generate_transaction(self, transaction_id, start_date):
        """Generate a single delivery transaction"""
        # Select random cities for pickup and delivery
        pickup_city = random.choice(list(self.cities.keys()))
        delivery_city = random.choice(list(self.cities.keys()))
        
        # Generate coordinates
        pickup_lat, pickup_lon = self.generate_coordinates(pickup_city)
        delivery_lat, delivery_lon = self.generate_coordinates(delivery_city)
        
        # Calculate distance (approximate)
        distance = np.sqrt(
            (delivery_lat - pickup_lat)**2 + 
            (delivery_lon - pickup_lon)**2
        ) * 111  # Convert to kilometers
        
        # Select priority and vehicle
        priority = random.choice(self.priority_levels)
        vehicle_type = random.choice(list(self.vehicle_types.keys()))
        vehicle_capacity = self.vehicle_types[vehicle_type]
        
        # Generate timestamps
        pickup_time = start_date
        traffic_multiplier = self.get_traffic_multiplier(pickup_time, pickup_city)
        
        # Adjust delivery time based on traffic and distance
        base_delivery_time = distance * 0.5  # 0.5 hours per km
        adjusted_delivery_time = base_delivery_time * traffic_multiplier
        
        estimated_delivery = pickup_time + timedelta(hours=adjusted_delivery_time)
        earliest_delivery, latest_delivery = self.generate_time_window(pickup_time, priority)
        
        # Add potential delay (higher probability during monsoon)
        delay_probability = 0.15
        if pickup_time.month in self.monsoon_months:
            delay_probability = 0.35
            
        if random.random() < delay_probability:
            actual_delay = random.uniform(1, 3) * traffic_multiplier
            actual_delivery = estimated_delivery + timedelta(hours=actual_delay)
        else:
            actual_delivery = estimated_delivery + timedelta(minutes=random.uniform(-30, 30))
        
        # Generate package details
        weight = round(random.uniform(0.1, vehicle_capacity['capacity_weight'] * 0.8), 2)
        volume = round(random.uniform(0.001, vehicle_capacity['capacity_volume'] * 0.8), 3)
        
        # Calculate price (in INR)
        base_price = (distance * 15) + (weight * 10) + (volume * 100)  # Base price in INR
        priority_multipliers = {
            'Standard': 1.0,
            'Next Day': 1.3,
            'Express': 1.6,
            'Same Day': 2.0,
            'Premium': 2.5
        }
        final_price = round(base_price * priority_multipliers[priority], 2)
        
        # Determine status
        current_time = datetime.now()
        if actual_delivery > current_time:
            if pickup_time > current_time:
                status = 'Pending'
            else:
                status = 'In Transit'
        else:
            if random.random() < 0.05:  # 5% chance of failure
                status = 'Failed'
            else:
                status = 'Delivered'
        
        return {
            'transaction_id': transaction_id,
            'pickup_city': pickup_city,
            'delivery_city': delivery_city,
            'pickup_lat': pickup_lat,
            'pickup_lon': pickup_lon,
            'delivery_lat': delivery_lat,
            'delivery_lon': delivery_lon,
            'distance_km': round(distance, 2),
            'weight_kg': weight,
            'volume_m3': volume,
            'pickup_time': pickup_time,
            'estimated_delivery': estimated_delivery,
            'actual_delivery': actual_delivery,
            'earliest_delivery': earliest_delivery,
            'latest_delivery': latest_delivery,
            'status': status,
            'priority': priority,
            'vehicle_type': vehicle_type,
            'vehicle_capacity_weight': vehicle_capacity['capacity_weight'],
            'vehicle_capacity_volume': vehicle_capacity['capacity_volume'],
            'payment_method': random.choice(self.payment_methods),
            'price_inr': final_price,
            'traffic_multiplier': round(traffic_multiplier, 2),
            'is_monsoon': pickup_time.month in self.monsoon_months
        }

    def generate_dataset(self, num_records, start_date=None, end_date=None):
        """Generate multiple delivery transactions"""
        if start_date is None:
            start_date = datetime.now() - timedelta(days=30)
        if end_date is None:
            end_date = datetime.now()
        
        time_range = (end_date - start_date).total_seconds()
        timestamps = [
            start_date + timedelta(seconds=random.uniform(0, time_range))
            for _ in range(num_records)
        ]
        timestamps.sort()
        
        transactions = []
        for i in range(num_records):
            transaction = self.generate_transaction(
                f'IN{str(i+1).zfill(6)}',  # Indian transaction ID format
                timestamps[i]
            )
            transactions.append(transaction)
        
        return pd.DataFrame(transactions)

    # def save_to_sqlite(self, df, db_path='indian_delivery_transactions.db'):
    #     """Save the generated data to SQLite database"""
    #     conn = sqlite3.connect(db_path)
    #     df.to_sql('delivery_transactions', conn, if_exists='replace', index=False)
        
    #     # Create indices for common query fields
    #     c = conn.cursor()
    #     c.execute('CREATE INDEX IF NOT EXISTS idx_transaction_id ON delivery_transactions(transaction_id)')
    #     c.execute('CREATE INDEX IF NOT EXISTS idx_status ON delivery_transactions(status)')
    #     c.execute('CREATE INDEX IF NOT EXISTS idx_pickup_time ON delivery_transactions(pickup_time)')
    #     c.execute('CREATE INDEX IF NOT EXISTS idx_cities ON delivery_transactions(pickup_city, delivery_city)')
    #     conn.commit()
    #     conn.close()

    def save_to_csv(self, df, file_path='indian_delivery_transactions.csv'):
        """Save the generated data to CSV file"""
        df.to_csv(file_path, index=False)

# Usage example
if __name__ == "__main__":
    generator = IndianDeliveryDataGenerator()
    
    # Generate 1000 delivery transactions from last 30 days
    df = generator.generate_dataset(
        num_records=60000,
        start_date=datetime.now() - timedelta(days=360),
        end_date=datetime.now() - timedelta(days=60)
    )
    
    # Save to both SQLite and CSV
    # generator.save_to_sqlite(df)
    generator.save_to_csv(df)

In [5]:
df = pd.read_csv("indian_delivery_transactions.csv")

In [7]:
pd.set_option('display.max_columns', None) 

In [8]:
df

,transaction_id,pickup_city,delivery_city,pickup_lat,pickup_lon,delivery_lat,delivery_lon,distance_km,weight_kg,volume_m3,pickup_time,estimated_delivery,actual_delivery,earliest_delivery,latest_delivery,status,priority,vehicle_type,vehicle_capacity_weight,vehicle_capacity_volume,payment_method,price_inr,traffic_multiplier,is_monsoon
0,IN000001,Chennai,Kolkata,13.113780,80.309601,22.620558,88.372234,1383.65,972.15,3.414,2024-02-07 01:06:46.391011,2024-03-18 21:05:44.662568,2024-03-18 20:52:30.831289,2024-02-07 09:06:46.391011,2024-02-08 01:06:46.391011,Delivered,Next Day,Mini Truck,2500,20.0,Digital Wallet,40063.01,1.42,False
1,IN000002,Mumbai,Delhi,19.038029,72.801687,28.563247,77.281436,1168.39,348.80,2.510,2024-02-07 01:12:08.966565,2024-03-15 14:53:41.052960,2024-03-15 15:17:00.794059,2024-02-08 01:12:08.966565,2024-02-09 01:12:08.966565,Delivered,Standard,Mahindra Pickup,1200,12.0,UPI,21264.89,1.54,False
2,IN000003,Chennai,Kolkata,13.008907,80.294746,22.551902,88.392082,1389.21,307.60,2.006,2024-02-07 01:23:31.227104,2024-03-13 14:03:27.442155,2024-03-13 13:49:56.476180,2024-02-07 09:23:31.227104,2024-02-08 01:23:31.227104,Delivered,Next Day,Mahindra Pickup,1200,12.0,Digital Wallet,31349.20,1.23,False
3,IN000004,Hyderabad,Delhi,17.374393,78.574246,28.687724,77.198214,1265.03,1230.25,12.075,2024-02-07 01:25:45.623224,2024-03-16 04:34:24.258281,2024-03-16 04:25:38.963851,2024-02-07 09:25:45.623224,2024-02-08 01:25:45.623224,Delivered,Next Day,Mini Truck,2500,20.0,Net Banking,42231.17,1.45,False
4,IN000005,Kolkata,Mumbai,22.480900,88.368970,19.006689,72.945940,1754.85,545.75,1.599,2024-02-07 01:26:26.139326,2024-04-07 07:30:11.409007,2024-04-07 07:53:48.267264,2024-02-07 02:26:26.139326,2024-02-07 05:26:26.139326,Delivered,Same Day,Mahindra Pickup,1200,12.0,Credit Card,63880.40,1.65,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,IN059996,Delhi,Pune,28.638015,77.157647,18.600717,73.850833,1173.05,80.73,5.319,2024-12-03 00:28:30.413235,2025-01-07 16:13:54.901902,2025-01-07 16:29:51.159444,2024-12-03 01:28:30.413235,2024-12-03 03:28:30.413235,Delivered,Premium,Tata Ace,750,8.0,Net Banking,47337.25,1.46,False
59996,IN059997,Kolkata,Kolkata,22.484788,88.322657,22.512633,88.324940,3.10,1523.16,16.616,2024-12-03 00:31:04.095954,2024-12-03 02:51:14.513856,2024-12-03 03:08:42.054548,2024-12-03 01:31:04.095954,2024-12-03 03:31:04.095954,Delivered,Premium,Large Truck,4000,32.0,Digital Wallet,42349.30,1.51,False
59997,IN059998,Delhi,Chennai,28.682119,77.262440,13.137420,80.230031,1756.62,1885.82,7.737,2024-12-03 00:36:10.094097,2025-02-05 22:03:40.513956,2025-02-05 22:09:43.298880,2024-12-03 01:36:10.094097,2024-12-03 04:36:10.094097,In Transit,Same Day,Mini Truck,2500,20.0,Digital Wallet,91962.49,1.77,False
59998,IN059999,Hyderabad,Pune,17.407092,78.568144,18.546545,73.781148,546.20,227.59,3.429,2024-12-03 00:50:21.052310,2024-12-18 10:27:28.379421,2024-12-18 10:07:49.215546,2024-12-04 00:50:21.052310,2024-12-05 00:50:21.052310,Delivered,Standard,Cargo Auto,500,5.0,Debit Card,10811.83,1.35,False
